<a href="https://colab.research.google.com/github/FranciscoFoz/7-days-of-GitHub/blob/main/Aprendizado%20Supervisionado/Classifica%C3%A7%C3%A3o/15_Classificacao_Otimizacao_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso Alura - Classificação: otimizando modelos

## 1. Construindo os modelos iniciais

In [ ]:
import pandas as pd

In [ ]:
url_dados = 'https://3070-classificacao-otimizacao.s3.us-east-2.amazonaws.com/dados_inadimplencia.csv'

In [ ]:
dados = pd.read_csv(url_dados)
dados.head()

In [ ]:
dados.describe()

In [ ]:
dados.inadimplente.value_counts()

In [ ]:
x = dados.drop('inadimplente', axis =1)
y = dados['inadimplente']

In [ ]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.33, random_state=RANDOM_STATE, stratify=y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

modelo_dt = DecisionTreeClassifier(max_depth=3, random_state=RANDOM_STATE)
modelo_dt.fit(x_treino, y_treino)

from sklearn.metrics import recall_score

recall_dt = recall_score(y_teste, modelo_dt.predict(x_teste))
print(f"Recall do DT = {recall_dt:.3f}")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

logistic_pipeline = make_pipeline(StandardScaler(), LogisticRegression())

logistic_pipeline.fit(x_treino, y_treino)

recall_lr = recall_score(y_teste, logistic_pipeline.predict(x_teste))
print(f"Recall do logistic regression = {recall_lr:.3f}")

## 2. Realizando a busca em grade

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

param_grid_dt = {
    'criterion':  ['gini', 'entropy'],
    'max_depth': np.linspace(6, 12, 4, dtype=int),
    'min_samples_split': np.linspace(5, 20, 4, dtype=int),
    'min_samples_leaf': np.linspace(5, 20, 4, dtype=int),
    'max_features': ['sqrt', 'log2'],
    'splitter': ['best', 'random']
}



cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

dt_grid_search = GridSearchCV(estimator = DecisionTreeClassifier(random_state=RANDOM_STATE),
                              param_grid = param_grid_dt,
                              scoring = "recall",
                              n_jobs = -1,
                              cv = cv)

dt_grid_search.fit(x_treino, y_treino)

In [ ]:

dt_grid_search.best_params_

In [ ]:
df_cv_results_dt = pd.DataFrame(dt_grid_search.cv_results_)

df_cv_results_dt.loc[[dt_grid_search.best_index_]]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(
    df_cv_results_dt['param_max_depth'],
    df_cv_results_dt['mean_test_score'],
    )

plt.title('max_depth vs. mean_test_score')
plt.xlabel('max_depth')
plt.ylabel('recall')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(
    df_cv_results_dt['param_min_samples_split'],
    df_cv_results_dt['mean_test_score'],
    )

plt.title('min_samples_split vs. mean_test_score')
plt.xlabel('min_samples_split')
plt.ylabel('recall')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(
    df_cv_results_dt['param_min_samples_leaf'],
    df_cv_results_dt['mean_test_score'],
    )

plt.title('min_samples_leaf vs. mean_test_score')
plt.xlabel('min_samples_leaf')
plt.ylabel('recall')
plt.show()

In [ ]:
max_iter = np.linspace(100, 300, 5, dtype=int)
c = [0.001, 0.01, 0.1, 1, 10]

param_grid_lr = [
    {'logisticregression__solver' : ['newton-cg', 'lbfgs'],
      'logisticregression__penalty' : ['l2'],
      'logisticregression__max_iter' : max_iter,
       'logisticregression__C' : c},
    {'logisticregression__solver' : ['liblinear'],
      'logisticregression__penalty' : ['l1', 'l2'],
      'logisticregression__max_iter' : max_iter,
      'logisticregression__C' : c},
]

lr_grid_search = GridSearchCV(estimator = make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
                               param_grid = param_grid_lr,
                               scoring = "recall",
                               n_jobs = -1,
                               cv = cv)

lr_grid_search.fit(x_treino, y_treino)

In [ ]:
lr_grid_search.best_params_

In [ ]:
df_cv_results_lr = pd.DataFrame(lr_grid_search.cv_results_)
df_cv_results_lr.head()

In [ ]:
df_cv_results_lr.loc[[lr_grid_search.best_index_]]

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(
    df_cv_results_lr['param_logisticregression__max_iter'],
    df_cv_results_lr['mean_test_score'],
    )

plt.title('max_iter vs. mean_test_score')
plt.xlabel('max_iter')
plt.ylabel('recall')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(
    df_cv_results_lr['param_logisticregression__C'],
    df_cv_results_lr['mean_test_score'],
    )

plt.title('logisticregression__C vs. mean_test_score')
plt.xlabel('logisticregression__C')
plt.ylabel('recall')
plt.show()

In [ ]:
from numpy import linspace
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Definindo a grade de hiperparâmetros
param_grid_knn = {
    'knn__n_neighbors': linspace(5, 25, 10, dtype=int),
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

pipe_knn = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

knn_grid_search = GridSearchCV(estimator = pipe_knn,
                               param_grid = param_grid_knn,
                               scoring = "recall",
                               n_jobs = -1,
                               cv = cv)

knn_grid_search.fit(x_treino, y_treino)

In [ ]:
knn_grid_search.best_params_

## 3. Validação Cruzada Aninhada

In [ ]:
inner_cv = StratifiedKFold(shuffle=True, random_state=RANDOM_STATE)

outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

In [ ]:
from sklearn.model_selection import cross_val_score

dt_nested_gs = GridSearchCV(estimator = DecisionTreeClassifier(random_state=RANDOM_STATE),
                                         param_grid = param_grid_dt,
                                         scoring = "recall",
                                         n_jobs = -1,
                                         cv = inner_cv
                                         )

dt_nested_scores = cross_val_score(dt_nested_gs, x_treino, y_treino, cv=outer_cv)

In [ ]:
print(f'resultado de cada iteração do cv externo: {dt_nested_scores}')
print(f'média: {dt_nested_scores.mean()}')

## 4. Busca Aleatória